Explanation of this code

http://www.paulvangent.com/2016/04/01/emotion-recognition-with-python-opencv-and-a-face-dataset/

# 1. Labeling Dataset

In [1]:
import cv2
import random
import numpy as np
from shutil import copyfile
import glob

emotions = ["neutral", "anger", "contempt", "disgust", "fear", "happy", "sadness", "surprise"] #Define emotion order
#participants = glob.glob("source_emotion/*") #Returns a list of all folders with participant numbers

emotion = 'surprise'

In [2]:
SIZE_FACE = 64

data = {}

def get_files(emotion): #Define function to get file list, randomly shuffle it and split 80/20
    files = glob.glob("dataset/%s/*" %emotion)
    random.shuffle(files)
    training = files[:int(len(files)*0.8)] #get first 80% of file list
    prediction = files[-int(len(files)*0.2):] #get last 20% of file list
    return training, prediction

#img_preprocessing
#http://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_filtering/py_filtering.html

def make_sets():
    training_data = []
    training_labels = []
    prediction_data = []
    prediction_labels = []
    for emotion in emotions:
        training, prediction = get_files(emotion)
        #Append data to training and prediction list, and generate labels 0-7
        for item in training:
            image = cv2.imread(item) #open image
            img_blur = cv2.bilateralFilter(image, 9, 75, 75)
            gray = cv2.cvtColor(img_blur, cv2.COLOR_BGR2GRAY) #convert to grayscale
            img_norm = cv2.resize(gray, (SIZE_FACE, SIZE_FACE), interpolation = cv2.INTER_CUBIC) / 255. #normalization
            training_data.append(img_norm) #append image array to training data list
            training_labels.append(emotions.index(emotion))
    
        for item in prediction: #repeat above process for prediction set
            image = cv2.imread(item)
            img_blur = cv2.bilateralFilter(image, 9, 75, 75)
            gray = cv2.cvtColor(img_blur, cv2.COLOR_BGR2GRAY)
            img_norm = cv2.resize(gray, (SIZE_FACE, SIZE_FACE), interpolation = cv2.INTER_CUBIC) / 255. #normalization
            prediction_data.append(img_norm)
            prediction_labels.append(emotions.index(emotion))

    return training_data, training_labels, prediction_data, prediction_labels

training_data, training_labels, prediction_data, prediction_labels = make_sets()

In [3]:
#Convert all data into numpy
X, Y, X_test, Y_test = np.array(training_data), np.array(training_labels), np.array(prediction_data), np.array(prediction_labels)

In [6]:
a = X[0]
a.shape

(64, 64)

In [7]:
#Show image

from scipy.misc import toimage
toimage(a).show()

# 2. Training with TF learn (alexnet)

In [4]:
from __future__ import division, print_function, absolute_import

import tflearn
from tflearn.data_utils import shuffle, to_categorical
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.estimator import regression
from tflearn.data_preprocessing import ImagePreprocessing
from tflearn.data_augmentation import ImageAugmentation

#import tflearn
#from tflearn.layers.core import input_data, dropout, fully_connected
#from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.normalization import local_response_normalization
from tflearn.layers.estimator import regression

In [5]:
#Shuffling and one hot encoding

X, Y = shuffle(X,Y)
X_test, Y_test = shuffle(X_test, Y_test)
#Y = to_categorical(Y, 8)
#Y_test = to_categorical(Y_test, 8)

In [6]:
#Just for checking
X = X[0:500, ]
Y = Y[0:500, ]
X_test = X[0:100, ]
Y_test = Y[0:100, ]

# Data shuffle code

#Shuffle the data

def randomize(dataset, labels):
    permutation = np.random.permutation(labels.shape[0])
    shuffled_dataset = dataset[permutation,:,:]
    shuffled_labels = labels[permutation]
    return shuffled_dataset, shuffled_labels

X, Y = randomize(X, Y)
testX, testY = randomize(testX, testY)
#test_dataset, test_labels = randomize(test_dataset, test_labels)
#valid_dataset, valid_labels = randomize(valid_dataset, valid_labels)

In [7]:
def dense_to_one_hot(labels_dense, num_classes=8):
  """Convert class labels from scalars to one-hot vectors."""
  num_labels = labels_dense.shape[0]
  index_offset = np.arange(num_labels) * num_classes
  labels_one_hot = np.zeros((num_labels, num_classes))
  labels_one_hot.flat[index_offset + labels_dense.ravel()] = 1
  return labels_one_hot

Y = dense_to_one_hot(Y)
Y_test = dense_to_one_hot(Y_test)

In [8]:
#Convert dataset into...
IMAGE_SIZE = 64

X = X.reshape([-1, IMAGE_SIZE, IMAGE_SIZE, 1])
X_test = X_test.reshape([-1, IMAGE_SIZE, IMAGE_SIZE, 1])

In [9]:
# Building convolutional network
network = input_data(shape=[None, IMAGE_SIZE, IMAGE_SIZE, 1], name='input')
network = conv_2d(network, 64, 3, activation='relu', regularizer="L2")
network = max_pool_2d(network, 2)
network = local_response_normalization(network)
network = conv_2d(network, 128, 3, activation='relu', regularizer="L2")
network = max_pool_2d(network, 2)
network = local_response_normalization(network)
network = fully_connected(network, 256, activation='tanh')
network = dropout(network, 0.8)
network = fully_connected(network, 512, activation='tanh')
network = dropout(network, 0.8)
network = fully_connected(network, 8, activation='softmax')
network = regression(network, optimizer='adam', learning_rate=0.01,
                     loss='categorical_crossentropy', name='target')

# Training
model = tflearn.DNN(network, tensorboard_verbose=0)
model.fit({'input': X}, {'target': Y}, n_epoch=20,
           validation_set=({'input': X_test}, {'target': Y_test}),
           snapshot_step=100, show_metric=True, run_id='convnet_mnist')

Training Step: 160  | total loss: 2.57031
| Adam | epoch: 020 | loss: 2.57031 - acc: 0.1294 | val_loss: 2.22483 - val_acc: 0.1700 -- iter: 500/500
Training Step: 160  | total loss: 2.57031
| Adam | epoch: 020 | loss: 2.57031 - acc: 0.1294 | val_loss: 2.22483 - val_acc: 0.1700 -- iter: 500/500
--
